# Spectral features

In [1]:
#import necessary libraries
from mido import MidiFile, MidiTrack, Message
from mido.midifiles.meta import MetaMessage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import mido
import os
import math
import numpy as np
import pandas as pd
import statistics
from scipy.stats import entropy
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import itertools
%matplotlib inline

#  Feature Extraction

In [2]:
#Read all jigs
os.getcwd()
files_j = []
os.chdir("/home/girija/Documents/Project/jigs_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_j.append(file)
        
#get spectral features
chroma_stft = []
spectral_centroid = []
spectral_bandwidth = []
spectral_rolloff = []
zerocross_rate = []    
mfcc = []
for file in files_j:
    y, sr = librosa.load(file)
    chroma_stft.append(np.mean(librosa.feature.chroma_stft(y,sr)))
    spectral_bandwidth.append(np.mean(librosa.feature.spectral_bandwidth(y,sr)))
    spectral_centroid.append(np.mean(librosa.feature.spectral_centroid(y,sr)))
    spectral_rolloff.append(np.mean(librosa.feature.spectral_rolloff(y,sr)))
    zerocross_rate.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    mfcc.append(list(np.mean(librosa.feature.mfcc(y, sr),axis=1)))

In [3]:
#get mfcc coefficients
mfcc_transpose = list(np.transpose(mfcc))

In [4]:
names = []
for i in range(1,21):
    names.append('mfcc'+str(i))

In [5]:
#create dataframe for jigs
df_j = pd.DataFrame()
df_j['chroma_stft'] = chroma_stft
df_j['spectral_centroid'] = spectral_centroid
df_j['spectral_bandwidth'] = spectral_bandwidth
df_j['spectral_rolloff'] = spectral_rolloff
df_j['zerocross_rate'] = zerocross_rate
for i in range(0,20):
    df_j[names[i]] = mfcc_transpose[i]

In [6]:
#Read all reels
os.getcwd()
files_r = []
os.chdir("/home/girija/Documents/Project/reels_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_r.append(file)
   
#get spectral features
chroma_stftr = []
spectral_centroidr = []
spectral_bandwidthr = []
spectral_rolloffr = []
zerocross_rater = []    
mfccr = []
for file in files_r:
    y, sr = librosa.load(file)
    chroma_stftr.append(np.mean(librosa.feature.chroma_stft(y,sr)))
    spectral_bandwidthr.append(np.mean(librosa.feature.spectral_bandwidth(y,sr)))
    spectral_centroidr.append(np.mean(librosa.feature.spectral_centroid(y,sr)))
    spectral_rolloffr.append(np.mean(librosa.feature.spectral_rolloff(y,sr)))
    zerocross_rater.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    mfccr.append(list(np.mean(librosa.feature.mfcc(y, sr),axis=1)))

In [7]:
#get mfcc coefficients
mfcc_transposer = list(np.transpose(mfccr))

# Dataset Creation

In [8]:
#create dataframe for reels
df_r = pd.DataFrame()
df_r['chroma_stft'] = chroma_stftr
df_r['spectral_centroid'] = spectral_centroidr
df_r['spectral_bandwidth'] = spectral_bandwidthr
df_r['spectral_rolloff'] = spectral_rolloffr
df_r['zerocross_rate'] = zerocross_rater
for i in range(0,20):
    df_r[names[i]] = mfcc_transposer[i]

In [9]:
#create dataset consisting of both Jigs and Reels
df = pd.concat([df_j,df_r])
df = df.reset_index(drop=True)
df.tail()

chroma_stft  spectral_centroid  spectral_bandwidth  spectral_rolloff  \
195     0.148660        1031.328753         1072.148084       1656.137246   
196     0.163704        1064.109903         1087.510260       1713.873937   
197     0.158861        1079.670193         1072.330862       1714.041520   
198     0.159380         982.270248         1039.383557       1694.910524   
199     0.174688         965.110852          988.119455       1608.386340   

     zerocross_rate       mfcc1       mfcc2      mfcc3      mfcc4      mfcc5  \
195        0.052690 -436.289577  140.128521 -21.334249   9.057607 -14.849008   
196        0.053514 -422.807628  134.243412 -28.255029   7.091049 -19.390896   
197        0.056870 -432.714635  132.208581 -28.358650   2.597679 -16.003192   
198        0.043111 -461.964452  127.331665 -21.525253   8.991480  -0.424482   
199        0.052533 -383.306926  164.200017 -33.226127  11.111880  -6.723910   

     ...     mfcc11    mfcc12    mfcc13    mfcc14    mfcc15    mfcc16  \
195  ...   0.772349  5.009435  4.682262 -0.122849 -3.549157  0.701118   
196  ...  -0.850560  8.209950  6.455159  2.961727 -4.624097 -9.484302   
197  ...   2.835687  1.023015 -2.198213 -4.649535 -5.980591 -3.197716   
198  ... -13.791054 -7.392597 -3.756143 -1.721635 -4.405188 -6.131096   
199  ...  -9.534174 -4.013999 -3.869015 -2.948175 -4.411552 -4.978004   

        mfcc17    mfcc18    mfcc19     mfcc20  
195   6.121475  1.219697 -9.623463 -10.077767  
196 -11.330936 -9.063894 -7.667550  -5.393953  
197  -0.233495  0.067835 -1.866109  -1.362359  
198  -2.987074  1.105258  1.745829   0.504778  
199  -3.715087 -1.648026 -1.580535  -2.697511  

[5 rows x 25 columns]

In [11]:
df_combined = pd.concat([df_j,df_r],axis=1)

In [12]:
#create dataset with input format that is compatible with K-Means
dataset = []
for idx, rows in df_combined.iterrows():
    dataset.append([rows.chroma_stft, rows.spectral_centroid, rows.zerocross_rate, rows.mfcc1, rows.mfcc2,
       rows.mfcc3,rows.mfcc4, rows.mfcc5, rows.mfcc6, rows.mfcc7, rows.mfcc8, rows.mfcc9, rows.mfcc10,
       rows.mfcc11, rows.mfcc12, rows.mfcc13, rows.mfcc14, rows.mfcc15, rows.mfcc16, rows.mfcc17,
       rows.mfcc18, rows.mfcc19])

# K-Means Clustering

In [13]:
def KMeansClustering(dataset, k, max_itr):
    #create an empty list of centroids
    centroids = []
    #select first 'k' datapoints as the centroids
    for i in range(k):
        centroids.append(dataset[i])
    
    for itr in range(max_itr):
        clusters = []
        #find distance between a data-point and each centroid
        for point in dataset:
            distances = [np.linalg.norm(np.asarray(point)-np.asarray(centroid)) for centroid in centroids]
            #select centroid at minimum distance
            clusters.append(distances.index(min(distances)))

        #empty dictionary to store data-points as per cluster numbers    
        data = {}
        for i in range(k):
            data[i] = []
        
        #append data-points to the list based on cluster number    
        for i in range(len(dataset)):
            data[clusters[i]].append(dataset[i])
        
        #update centroids based on mean of data-points in the cluster    
        centroids = []
        for point in data:
            centroids.append(np.mean(data[point],axis = 0))
            
    return(clusters)

# Silhouette Analysis

In [14]:
#Evaluation using silhouette analysis
def silhouette_analysis(datasets,k, clusters):
    a = []
    intra_cluster = []
    for i in range(k):
        tmp = []
        #find tunes belonging to a particular cluster
        for j in range(len(datasets)):
            if(i==clusters[j]):
                tmp.append(j)
        intra_cluster.append(tmp)
    
    #find intra-cluster distances of all data-points
    for cluster in intra_cluster:
        for i in range(len(cluster)):
            other_points = cluster[:i]+cluster[i+1:]
            dist = []
            flag = 0
            for point in other_points:
                if(len(other_points)>1):
                    dist.append(np.linalg.norm(np.asarray(datasets[point])-np.asarray(datasets[cluster[i]])))
                else:
                    dist.append(np.mean(datasets[point]))
            a.append(np.mean(dist))
    
    #find inter-cluster distances of all data-points
    inter_clusters = []
    for i in range(len(intra_cluster)):
        inter_clusters.append(list(itertools.chain.from_iterable(intra_cluster[:i]+intra_cluster[i+1:])))
    
    b = []
    for i in range(len(intra_cluster)):
        for j in range(len(inter_clusters)):
            if(i==j):
                for given_point in intra_cluster[i]:
                    dist = []
                    for other_point in inter_clusters[j]:
                        dist.append(np.linalg.norm(np.asarray(datasets[given_point])-np.asarray(datasets[other_point])))
                    b.append(min(dist))
    
    #find silhouette co-efficient of each point
    result = []
    for i in range(len(a)):
        for j in range(len(b)):
            if(i==j):
                res = (b[i]-a[i])/(max(a[i],b[i]))
                if(not math.isnan(res)):
                    result.append(res)
    return(statistics.mean(result))

# Result of Spectral Features

In [15]:
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 100)
    total+=silhouette_analysis(dataset,4, clusters)

In [16]:
total

-0.21381155373730146